#### Imports

In [48]:
import pyspark
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import udf ,col , split, max, min, sum,year,month,dayofmonth,weekofyear
from pyspark.sql.types import DateType,StringType
from datetime import datetime

# Basic operations

In [3]:
spark = SparkSession.builder.appName("stocksly").getOrCreate()
spark

25/02/17 18:23:31 WARN Utils: Your hostname, ayushkhaire resolves to a loopback address: 127.0.1.1; using 192.168.2.14 instead (on interface wlp2s0)
25/02/17 18:23:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/17 18:23:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
stocks = spark.read.csv(
    "../data_sourcing/source/stocks.csv", 
    header=True, 
    inferSchema=True
)

In [5]:
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|
+---+---------+-------------------+-----+-----+-----+-----+------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|
+---+---------+-------------------+-----+-----+-----+-----+------+
only showing top 5 rows



25/02/17 18:25:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [6]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- stockname: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)



In [7]:
stocks.select("stockname").show(5)

+---------+
|stockname|
+---------+
|    PLTR |
|    PLTR |
|    PLTR |
|    PLTR |
|    PLTR |
+---------+
only showing top 5 rows



In [8]:
stocks.select(
    ["stockname","open", "close"]
).show(5)

+---------+-----+-----+
|stockname| open|close|
+---------+-----+-----+
|    PLTR |81.94|81.44|
|    PLTR |81.39|82.01|
|    PLTR |81.96|82.13|
|    PLTR |82.19|82.38|
|    PLTR | 82.4|82.32|
+---------+-----+-----+
only showing top 5 rows



In [9]:
stocks.filter(
    stocks.stockname == "TSLA"
).show(5)

25/02/17 18:25:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+------+---------+-------------------+------+------+------+------+------+
|   _c0|stockname|          timestamp|  open|  high|   low| close|volume|
+------+---------+-------------------+------+------+------+------+------+
|354692|     TSLA|2024-12-23 09:00:00| 431.6| 434.2|431.28|434.19|  NULL|
|354693|     TSLA|2024-12-23 09:01:00|434.19|435.13|433.53| 434.5|  NULL|
|354694|     TSLA|2024-12-23 09:02:00|434.63|434.86|433.02| 433.7|  NULL|
|354695|     TSLA|2024-12-23 09:03:00|433.69|433.97|433.29| 433.6|  NULL|
|354696|     TSLA|2024-12-23 09:04:00|433.57| 433.6|432.61|433.28|  NULL|
+------+---------+-------------------+------+------+------+------+------+
only showing top 5 rows



In [10]:
stocks.filter(
    (
        (stocks.stockname == "TSLA") |
        (stocks.stockname == "PLTR")
    ) & (
    stocks.timestamp == "2024-12-23 09:00:00")
).show()

25/02/17 18:25:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+-------+---------+-------------------+-----+-----+------+------+------+
|    _c0|stockname|          timestamp| open| high|   low| close|volume|
+-------+---------+-------------------+-----+-----+------+------+------+
| 354692|     TSLA|2024-12-23 09:00:00|431.6|434.2|431.28|434.19|  NULL|
|5149233|     PLTR|2024-12-23 09:00:00|81.94|81.95| 81.16| 81.44|   0.0|
+-------+---------+-------------------+-----+-----+------+------+------+



In [11]:
stocks.filter(
    stocks.stockname.isin("TSLA", "PLTR") &
    (stocks.timestamp == "2024-12-23 09:00:00")
).show()

25/02/17 18:25:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+-------+---------+-------------------+-----+-----+------+------+------+
|    _c0|stockname|          timestamp| open| high|   low| close|volume|
+-------+---------+-------------------+-----+-----+------+------+------+
| 354692|     TSLA|2024-12-23 09:00:00|431.6|434.2|431.28|434.19|  NULL|
|5149233|     PLTR|2024-12-23 09:00:00|81.94|81.95| 81.16| 81.44|   0.0|
+-------+---------+-------------------+-----+-----+------+------+------+



In [12]:
stocks = stocks.withColumn(
    "date", split(
        col("timestamp"), 
    " ")[0]
    )
stocks = stocks.withColumn(
    "time", split(
        col("timestamp"), 
    " ")[1]
    )
stocks.show(5)

25/02/17 18:26:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+
only showing top 5 rows



In [13]:
date_parser = udf(
    lambda date : datetime.strptime(date, "%Y-%m-%d"),DateType()
)

In [14]:
stocks = stocks.withColumn(
    "new_date",
    date_parser(stocks.date)  
)

stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/17 18:26:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [15]:
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/17 18:26:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [16]:
def upper_turner(stockname):
    return stockname.upper()

upper_turner = udf(
    upper_turner, StringType()
)

In [17]:
stocks = stocks.withColumn(
    "stockname",upper_turner(stocks.stockname)
)
stocks.show(5)

+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|_c0|stockname|          timestamp| open| high|  low|close|volume|      date|    time|  new_date|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
|  0|    PLTR |2024-12-23 09:00:00|81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024-12-23|
|  1|    PLTR |2024-12-23 09:01:00|81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024-12-23|
|  2|    PLTR |2024-12-23 09:02:00|81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024-12-23|
|  3|    PLTR |2024-12-23 09:03:00|82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024-12-23|
|  4|    PLTR |2024-12-23 09:04:00| 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024-12-23|
+---+---------+-------------------+-----+-----+-----+-----+------+----------+--------+----------+
only showing top 5 rows



25/02/17 18:26:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , stockname, timestamp, open, high, low, close, volume
 Schema: _c0, stockname, timestamp, open, high, low, close, volume
Expected: _c0 but found: 
CSV file: file:///home/ayushkhaire/code/dataennginneerinng/stocksly/data_sourcing/source/stocks.csv


In [18]:
stocks.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- stockname: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- new_date: date (nullable = true)



In [19]:
new_stocks = stocks.select(
    "stockname",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "new_date",
    "time"
)

In [20]:
new_stocks.show(5)

+---------+-----+-----+-----+-----+------+----------+--------+
|stockname| open| high|  low|close|volume|  new_date|    time|
+---------+-----+-----+-----+-----+------+----------+--------+
|    PLTR |81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|
|    PLTR |81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|
|    PLTR |81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|
|    PLTR |82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|
|    PLTR | 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|
+---------+-----+-----+-----+-----+------+----------+--------+
only showing top 5 rows



In [21]:
new_stocks.describe().show()

25/02/17 18:26:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+------------------+------------------+------------------+------------------+-----------------+--------+
|summary|stockname|              open|              high|               low|             close|           volume|    time|
+-------+---------+------------------+------------------+------------------+------------------+-----------------+--------+
|  count| 69242798|          63772250|          63772250|          63772250|          63772250|         60931060|69242798|
|   mean|     NULL|157.51115919261184|157.55508494635941| 157.4645094963356|157.51097045883867|55927.47375937658|    NULL|
| stddev|     NULL|3076.7758426276955|3076.7765233488894| 3076.775057143922|3076.7758180089277|7578621.679077058|    NULL|
|    min|      AAL| 0.002199999988079| 0.002199999988079|0.0020999999251216|0.0020999999251216|              0.0|00:00:00|
|    max|      ZK |     109110.390625|     109110.390625|     109110.390625|     109110.390625|   4.590406656E10|23:59:10|
+-------+-------

# Basic stock analysis

In [27]:
new_stocks.groupBy("stockname").max("Open").show(15)

+---------+------------------+
|stockname|         max(Open)|
+---------+------------------+
|    ACHR |  11.6099996566772|
|    GRAB |  5.03000020980835|
|     LMND| 46.93000030517578|
|      CRS|            220.12|
|    PSTG |              65.8|
|    OMAB |             72.85|
|      CXM| 9.484999656677246|
|      NN |             18.54|
|     CVNA|            264.85|
|    BTSG |             19.99|
|     ODD |             51.01|
|     AFRM| 76.78990173339844|
|     DYN |             25.99|
| DOGE-USD|0.4332814216613769|
|   LNVGY |  27.0599994659424|
+---------+------------------+
only showing top 15 rows



In [28]:
new_stocks.groupBy("stockname").max("Open").withColumnRenamed("max(Open)", "MaxStockPrice").show(15)

+---------+------------------+
|stockname|     MaxStockPrice|
+---------+------------------+
|    ACHR |  11.6099996566772|
|    GRAB |  5.03000020980835|
|     LMND| 46.93000030517578|
|      CRS|            220.12|
|    PSTG |              65.8|
|    OMAB |             72.85|
|      CXM| 9.484999656677246|
|      NN |             18.54|
|     CVNA|            264.85|
|    BTSG |             19.99|
|     ODD |             51.01|
|     AFRM| 76.78990173339844|
|     DYN |             25.99|
| DOGE-USD|0.4332814216613769|
|   LNVGY |  27.0599994659424|
+---------+------------------+
only showing top 15 rows



In [31]:
new_stocks.groupBy("stockname").agg(max("Open").alias("MaxStockPrice")).show(15)

+---------+------------------+
|stockname|     MaxStockPrice|
+---------+------------------+
|    ACHR |  11.6099996566772|
|    GRAB |  5.03000020980835|
|     LMND| 46.93000030517578|
|      CRS|            220.12|
|    PSTG |              65.8|
|    OMAB |             72.85|
|      CXM| 9.484999656677246|
|      NN |             18.54|
|     CVNA|            264.85|
|    BTSG |             19.99|
|     ODD |             51.01|
|     AFRM| 76.78990173339844|
|     DYN |             25.99|
| DOGE-USD|0.4332814216613769|
|   LNVGY |  27.0599994659424|
+---------+------------------+
only showing top 15 rows



In [36]:
new_stocks.groupBy("stockname").agg(
    max("open").alias("MaxStockPrice"),
    sum("volume").alias("TotalVolume")
).show(15)

+---------+------------------+--------------+
|stockname|     MaxStockPrice|   TotalVolume|
+---------+------------------+--------------+
|    ACHR |  11.6099996566772|          NULL|
|    GRAB |  5.03000020980835|          NULL|
|     LMND| 46.93000030517578|  6.20761353E8|
|      CRS|            220.12|  3.01765302E8|
|    PSTG |              65.8|          NULL|
|    OMAB |             72.85|          NULL|
|      CXM| 9.484999656677246|    5.241375E8|
|      NN |             18.54|          NULL|
|     CVNA|            264.85| 1.524912677E9|
|    BTSG |             19.99|          NULL|
|     ODD |             51.01|          NULL|
|     AFRM| 76.78990173339844| 2.207234976E9|
|     DYN |             25.99|          NULL|
| DOGE-USD|0.4332814216613769|5.062422688E10|
|   LNVGY |  27.0599994659424|          NULL|
+---------+------------------+--------------+
only showing top 15 rows



In [44]:
new_stocks = (new_stocks.withColumn("Year", year(new_stocks.new_date))
                                .withColumn("Month", month(new_stocks.new_date))
                                .withColumn("Day", dayofmonth(new_stocks.new_date))
                                .withColumn("Week", weekofyear(new_stocks.new_date))
                 )

In [45]:
new_stocks.show(5)

+---------+-----+-----+-----+-----+------+----------+--------+----+-----+---+----+
|stockname| open| high|  low|close|volume|  new_date|    time|Year|Month|Day|Week|
+---------+-----+-----+-----+-----+------+----------+--------+----+-----+---+----+
|    PLTR |81.94|81.95|81.16|81.44|  NULL|2024-12-23|09:00:00|2024|   12| 23|  52|
|    PLTR |81.39|82.02|81.37|82.01|  NULL|2024-12-23|09:01:00|2024|   12| 23|  52|
|    PLTR |81.96|82.16|81.95|82.13|  NULL|2024-12-23|09:02:00|2024|   12| 23|  52|
|    PLTR |82.19| 82.4|82.11|82.38|  NULL|2024-12-23|09:03:00|2024|   12| 23|  52|
|    PLTR | 82.4| 82.5| 82.3|82.32|  NULL|2024-12-23|09:04:00|2024|   12| 23|  52|
+---------+-----+-----+-----+-----+------+----------+--------+----+-----+---+----+
only showing top 5 rows



In [51]:
yearly = new_stocks.groupBy(
    ['stockname', 'Year']
).agg(
    max("open").alias("YearlHigh"), 
    min("open").alias("YearlyLow")
).show(15)

+---------+----+-----------------+-----------------+
|stockname|Year|        YearlHigh|        YearlyLow|
+---------+----+-----------------+-----------------+
|     GDS |2024| 22.5249996185303|             18.5|
|    ALAB |2024| 147.318298339844| 131.774993896484|
|     VIK |2024|            47.49|          44.0267|
|    XPEV |2024|          14.2266|          11.7969|
|    PENN |2024| 20.9300003051758|            17.84|
|    ASAN |2024|            23.13| 21.2399997711182|
|     AMCR|2025|            10.38|             9.28|
|    TSLA |2024| 487.917999267578|         285.0528|
|     WULF|2024|8.475000381469727|5.091599941253662|
|      CRS|2025|           220.12|           168.02|
|      KGC|2024|            10.08|9.005000114440918|
|     SMCI|2024|35.40999984741211|            29.55|
|      CVX|2025|           163.35|         142.0615|
|     BTSG|2024|             20.0|          15.9801|
|    APPN |2024|             37.0|             33.0|
+---------+----+-----------------+------------

In [52]:
monthly = new_stocks.groupBy(
    ['stockname', 'Year', 'Month']
).agg(
    max("Open").alias("MonthHigh"), min("Open").alias("MonthLow")
).show(15)

+---------+----+-----+------------------+------------------+
|stockname|Year|Month|         MonthHigh|          MonthLow|
+---------+----+-----+------------------+------------------+
|     VRT |2024|   12|            122.52|             113.5|
|    WRBY |2024|   12|  25.3400001525879|             22.76|
|      PBR|2024|   12|              13.7|12.579999923706056|
|     GOOG|2024|   12|202.69000244140625|          171.4523|
|     OXY |2024|   12|             49.79|            45.157|
|      PDD|2024|   12|           103.625|           92.7501|
|    IPGP |2024|   12|              78.4|             72.47|
|    AAPL |2024|   12|          266.1547|          225.0813|
|      BA |2024|   12|          188.6036|  166.610000610352|
|    PRIM |2024|   12|             82.99|  77.0800018310547|
|     IREN|2025|    1|13.604999542236328|  9.39900016784668|
|     SLG |2024|   12|              69.0|             64.16|
|     VYX |2024|   12|             14.55|              13.0|
|      NU |2024|   12|  

In [59]:
weekly = new_stocks.groupBy(
    ['stockname', 'Year', 'Week']
).agg(
    max("Open").alias("WeekHigh"), min("Open").alias("WeekLow")
)
weekly.show(15)

+---------+----+----+----------------+------------------+
|stockname|Year|Week|        WeekHigh|           WeekLow|
+---------+----+----+----------------+------------------+
|    NATL |2024|  52|           36.33|             30.55|
|    NBIS |2024|  52|29.2800006866455|  25.6700000762939|
|     RIG |2024|  52|            3.66|  3.40000009536743|
|     DVN |2024|  50|            33.9|             33.85|
|     TSM |2024|  52|        221.1243|             198.6|
|    SMTC |2024|  52|           68.23|  62.0200004577637|
|    PATH |2024|  52|           13.62|  12.7349996566772|
|     KHC |2024|  52|30.8199996948242|  30.2549991607666|
|     BTG |2024|  51|            2.72|              2.43|
|     HMC |2024|  51|           25.75|             23.43|
|      IQ |2024|  50|          2.2985|              2.26|
|      KHC|2025|   2|30.8799991607666|             28.51|
|       BA|2024|   1|        197.7372|170.82000732421875|
|     MARA|2024|   1|           18.27|16.639999389648438|
|      GDS|202

In [60]:
weekly.withColumn(
    "Spread", weekly['WeekHigh'] - weekly['WeekLow']
).show()

+---------+----+----+------------------+------------------+-------------------+
|stockname|Year|Week|          WeekHigh|           WeekLow|             Spread|
+---------+----+----+------------------+------------------+-------------------+
|    NATL |2024|  52|             36.33|             30.55|  5.779999999999998|
|    NBIS |2024|  52|  29.2800006866455|  25.6700000762939| 3.6100006103516016|
|     RIG |2024|  52|              3.66|  3.40000009536743| 0.2599999046325703|
|     DVN |2024|  50|              33.9|             33.85|0.04999999999999716|
|     TSM |2024|  52|          221.1243|             198.6|  22.52430000000001|
|    SMTC |2024|  52|             68.23|  62.0200004577637|  6.209999542236304|
|    PATH |2024|  52|             13.62|  12.7349996566772| 0.8850003433227993|
|     KHC |2024|  52|  30.8199996948242|  30.2549991607666| 0.5650005340575994|
|     BTG |2024|  51|              2.72|              2.43|0.29000000000000004|
|     HMC |2024|  51|             25.75|

# The End